# Building the Machine Learning Models

During EDA we found out that, with regards to statistical measures in this exercise, NBA centers from the US and EU are essentially the same. Put another way, the statistical measures for (which we will build our predictive models) are independant of the 'US or EU' column. 

In this notebook, we will build predictive models for: Points, Assists, Rebounds, Steals and Blocks. Given that this is probably the most useful part of the project in terms of delivering real world value, the descriptions and comments/justifications are less technical and shorter to not dilute the the results of the models and other findings. Let's begin!

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures, normalize
from sklearn.feature_selection import RFECV
from mlxtend.feature_selection import SequentialFeatureSelector as SFS 
from scipy.stats import boxcox
from statsmodels.stats.diagnostic import het_breuschpagan
plt.rcParams["figure.figsize"] = [18,10]

with open("C:/Users/Lukas Buteliauskas/Desktop/Springboard Projects/Capstone Project 1 - NBA Analytics/"
          "3. Exploratory Data Analysis/Player Data Filtered.csv", "r") as player_data_file:
    player_data_df=pd.read_csv(player_data_file)

player_data_df_undrafted=player_data_df[player_data_df["Draft Placing"].isnull()].reset_index(drop=True) #undrafted player data
player_data_df=player_data_df[player_data_df["Draft Placing"].isnull()==False].reset_index(drop=True) # drafted player data

player_data_og=player_data_df # keeping a copy of the input data (for drafted players)
player_data_df=player_data_df.loc[(player_data_df["eFG%"]!=0.0) & (player_data_df["eFG%"]!=100.0)] # filtering out 0 or 100 eFG%
player_data_df=player_data_df.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,13]].reset_index(drop=True) #dropping US or EU

# Dropping the 'US or EU' and 'Draft Placing' columns for undrafted player data.
player_data_df_undrafted=player_data_df_undrafted.iloc[:,[0,1,2,3,4,6,7,8,9,10,11,13]].reset_index(drop=True) 

C:\Users\Lukas Buteliauskas\Anaconda3_\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Given that undrafted players make up a sizeable amount of data, and are themselves a specific group of players, their data cannot be disregarded, but at the same time the Draft Rank cannot be reverse engineered. Hence, it makes sense to consider both groups as distinct, and attempt to build models for both seperately. We will begin with models for drafted players.

In [2]:
print(player_data_df.info())
print(player_data_df_undrafted.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3860 entries, 0 to 3859
Data columns (total 13 columns):
2PA              3860 non-null float64
3P               3860 non-null float64
AST              3860 non-null float64
Age              3860 non-null int64
BLK              3860 non-null float64
Draft Placing    3860 non-null float64
MP               3860 non-null float64
PF               3860 non-null float64
PTS              3860 non-null float64
STL              3860 non-null float64
TOV              3860 non-null float64
TRB              3860 non-null float64
eFG%             3860 non-null float64
dtypes: float64(12), int64(1)
memory usage: 392.1 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471 entries, 0 to 470
Data columns (total 12 columns):
2PA     471 non-null float64
3P      471 non-null float64
AST     471 non-null float64
Age     471 non-null int64
BLK     471 non-null float64
MP      471 non-null float64
PF      471 non-null float64
PTS     471 non-null flo

## Defining Custom Functions

In [3]:
""" This prints the R^2 statistic for training and testing, the cross validated R^2 mean, 95% CI as well as the MSE.
We also print 'n_test_print' rows showing the actual value, the value predicted by the model and the absolute difference. """
def performance_prints(clf, X_train, X_test, y_train, y_test, cv=5, n_test_print=15):
    print("Training performance (R^2): %.3f" % clf.score(X_train, y_train))
    print("Testing performance (R^2): %.3f" % clf.score(X_test, y_test))
    scores=cross_val_score(clf, X_train, y_train, cv=cv)
    print("R^2 cross val mean: %.3f" % (scores.mean()))
    print("95%% confidence interval for R^2: (%.3f, %.3f)" % (scores.mean()-scores.std()*2, scores.mean()+scores.std()*2))
    print("Mean Squared Error (MSE): %.3f\n\nSome test prints:" % mean_squared_error(y_test, y_pred))

    for idx, data in enumerate(zip(y_test, y_pred)): 
        if idx<n_test_print:
            print("Actual: %.2f\t  Predicted: %.2f\tDifference (absolute): %.3f" % (data[0], data[1], np.abs(data[0]-data[1])))
            
""" Residual Plot. Predicted values (y_pred) on the X-axis, residuals on the Y-axis. """
def my_resid_plot(y_test, y_pred, label_name="", standardized=False): # for showing us the residual plots
    residuals=y_test-y_pred
    if standardized is True:
        residuals/=np.std(residuals)
        plt.scatter(y_pred, residuals, color="black")
        plt.title("Residual Plot")
        plt.xlabel("Predicted " + label_name + " Values")
        plt.ylabel("Residuals")
        plt.axhline(y=0, linewidth=3, color="black")
        plt.show()
    else:
        plt.scatter(y_pred, residuals, color="black")
        plt.title("Residual Plot")
        plt.xlabel("Predicted " + label_name + " Values")
        plt.ylabel("Residuals")
        plt.axhline(y=0, linewidth=3, color="black")
        plt.show()

""" Residual plots, this time with the columns in the 'columns' parameter on the X-axis. """
def my_resid_plots(y_test, y_pred, X_test, columns): # residual plots for the features
    residuals=y_test-y_pred
    print("Heteroscendasticity hypothesis test p value:", het_breuschpagan(resid=residuals, exog_het=X_test)[1])
    for idx, column in enumerate(columns):
        plt.scatter(X_test[:,idx], residuals, color="black")
        plt.title("Residual Plot for "+ column)
        plt.xlabel(column + " Values")
        plt.ylabel("Residuals")
        plt.axhline(y=0, linewidth=3, color="black")
        plt.show() 

""" Plot of the Actual (observed) values on the X-axis and Predicted values on the Y-axis. """
def prediction_plot(y_test, y_pred, label_name=""): # for showing actual values vs predicted values
    plt.scatter(y_test, y_pred, color="black")
    plt.plot([x for x in range(0,int(np.max([np.max(y_test), np.max(y_pred)])))]) #straight line
    plt.title("Actual vs Predicted " + label_name + " Plot")
    plt.xlabel("Actual " + label_name + " Values")
    plt.ylabel("Predicted " + label_name + " Values")
    plt.show()
    
""" Apply the boxcox transformation to the 'data' parameter and return the transformed dataset. 'data' can be a Pandas dataframe
or a Numpy array. """
def my_normalizer(data, info_print=False):
    if isinstance(data, pd.DataFrame): # if 'data' is a dataframe
        new_df=data
        transformed_columns=[]
        for column in new_df.columns: # iterate over the columns in the dataframe
            if np.min(new_df[column]>=1) and column!="eFG%": # if the minimum value is greater than 1
                new_df[column]=boxcox(player_data_df[column].values)[0] # apply the boxcox transformation to the column
                transformed_columns.append(column) # and append it to the transformed_columns list
        if info_print is True:
            return new_df, transformed_columns
        else:
            return new_df
    else: # if 'data' is a multi-dimentional array
        new_array=data
        transformed_indices=[]
        for col_idx in range(new_array.shape[1]): # iterate over the columns (by cycling over the column index)
            if np.min(new_array[:,col_idx]>=1):
                new_array[:,col_idx]=boxcox(data[:,col_idx])[0]
                transformed_indices.append(col_idx)
        if info_print is True:
            return new_array, transformed_indices
        else:
            return new_array  
        
""" For real world applications, checking against new data."""
def get_predictions(data, clf, columns=feat_selector.support_, pol_degree=2):
    if isinstance(data, pd.DataFrame): #if 'data' is a dataframe 
        return clf.predict(PolynomialFeatures(degree=pol_degree).fit_transform(data)[:,columns])
    else: #if 'data' is a list or numpy array
        return clf.predict(PolynomialFeatures(degree=pol_degree).fit_transform(np.array(data))[:,columns])

""" Getting a mean absolute error for the predictions in certain intervals (only useful if heteroscendasticity is present)."""
""" CURRENTLY BROKEN, RETURNS ALL NaN """
def get_avg_error(y_test, y_pred, interval=1, simple_print=True):
    min_, max_=(np.min(y_pred), np.max(y_pred))
    n_steps=int((max_-min_)/interval)+1
    avg_abs_error=[]
    n_data_points=[]
    residuals=np.abs(y_test-y_pred) #absolute errors
    
    for step in range(n_steps): # go over each interval, calculate the avg absolute error and # data points in interval
        bool_indices=y_pred[(y_pred>=min_+step*interval) & (y_pred<=min_+(step+1)*interval)]
        avg_abs_error.append(np.mean(residuals[bool_indices]))
        n_data_points.append(np.sum(bool_indices))
    
    if simple_print is True:
        print(avg_abs_error)
    else:
        for idx, error in enumerate(avg_abs_error):
            print("[%f,%f] interval avg absolute error is %f, with %d data points" 
                % ((min_+idx*interval),(min_+(idx+1)*interval), error, n_data_points[idx]))

NameError: name 'feat_selector' is not defined

## Predictive Models for Drafted NBA Centers

### Points per Game (PTS column) Model

In [ ]:
X_eda=player_data_df[["Draft Placing", "eFG%", "Age", "MP"]] # creating a dataframe of only the features selected from EDA.
y=player_data_df["PTS"] # the dependant variable
X_train, X_test, y_train, y_test=train_test_split(X_eda, y, test_size=0.3, random_state=42)

clf_pts=LinearRegression().fit(X_train, y_train) #building/training the model
y_pred=clf_pts.predict(X_test) # predicting on the test set

performance_prints(clf_pts, X_train, X_test, y_train, y_test) #printing out the R^2 values for the training and testing sets
my_resid_plot(y_test, y_pred, label_name="PTS") # diagnosing the residuals of the model

**Technical evaluation comments.** Though the R^2 is quite high, the residual plot shows us that our model is biased. That is, the values of the errors in our predicitions (residuals) are not independant of the predicted values. Furthermore there is distinct increase in the spread of the residuals as well as a U-shape. All of the prior mean that perhaps we need to transform our data to achieve a model that fits the data better. The next logical step would be include polynomial transformations of the existing features. Given the scatter plots of the independant variables against PTS, we saw at most 1 point of inflection, this means we only need to consider 2nd order polynomials. Another consideration is the curse of dimentionality that could arise if we considered 3rd order polynomial features. Let us try this approach and see if our model performs better.

In [ ]:
X=PolynomialFeatures(degree=2).fit_transform(X_eda)
y=player_data_df["PTS"]

X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)
clf_pts=LinearRegression().fit(X_train, y_train)
y_pred=clf_pts.predict(X_test)

performance_prints(clf_pts, X_train, X_test, y_train, y_test)
my_resid_plot(y_test, y_pred, label_name="PTS")

**More technical commentary.** By introducing polynomial features we have succeeded in reducing bias (as demonstrated by a more random residual plot) and also reduced the MSE from 5.7 to 4.3, however **heteroscedasticity is still present** so we will need to deal with that at some point. Also, having computed 5-fold cross validation we can be confident in our model's ability to generalise to unseen data. We will now consider a larger subset of features from the dataframe (up to 2nd order polynomial features) and let the selector (RFECV) pick the best ones (the number of features is also chosen by RFECV).

In [ ]:
"""print("Columns used before (with MSE=3.2):", player_data_df.iloc[:,[2,3,4,5,6,9,10,11,12]].columns.values)
print("Columns used in the final version:", player_data_df.iloc[:,[3,5,6,10,12]].columns.values)"""
player_data_pts=player_data_df.iloc[:,[3,5,6,10,12]] 
X=PolynomialFeatures(degree=2).fit_transform(player_data_pts)
y=player_data_df["PTS"]

X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)
feat_selector=RFECV(LinearRegression(), cv=5).fit(X_train, y_train)
X_train, X_test, y_train, y_test=train_test_split(X[:,feat_selector.support_], y, test_size=0.3, random_state=42)

clf_pts=LinearRegression().fit(X_train, y_train)
y_pred=clf_pts.predict(X_test)

performance_prints(clf_pts, X_train, X_test, y_train, y_test)
my_resid_plot(y_test, y_pred, label_name="PTS")

Our model is on average 1.886ppg away from the true value (square root of MSE of 3.557). However, given that the variance in the error (residual) is not constant across the predicted value range, we have to be careful about our confidence in the predicted values. For example if we got a 4ppg (points per game) prediction for a certain player, we can be pretty certain that the maximum deviation from that value would be +-2.5ppg and an average that is much smaller than 1.886. Likewise if we got a prediction of 20ppg, this time the maximum deviation from the value would be +-7.5ppg and the average would be much greater than 1.886. 

The general point that we should be aware of, is that given the presence of heteroscendasticity (non-constant residual variance) and the fact that there are alot more 'average' players (0-10ppg) than 'above average players' (>10ppg), we cannot use average/means are reliable summary statistics (mean is sensitive to outliers and doesn't account for heteroscendasticity).

Of course, the statistic/graph of interest is dependant on the use case, but probably the most useful way to evaluate the prediction of a model is to provide the residual plot along with a box-plot, and the number of observations for each interval in predicted PTS values.

**Another technical point**. Heteroscendasticity is still present, so while this does not make our model unbiased it makes the inferences abit more difficult (as mentioned above). I've rescaled/normalized the data, tried weighted OLS and neither of those approaches introduced homoscendasticity. Finally I applied a boxcox transformation on the dependant variable (PTS) and while this managed to produce homoscendasticity, it ruined the interpretability/usefulness of the model as reversing the transformation of the predicted values yealds the heteroscendasticity once again. Hence, for the purposes of making the model useful, I've not transformed the dependant variable, but have been very careful to constantly signpost the presence of non-constant residual variance. Given that the models to come show similar problems, I will not go into as much detail for each model as I did here, but make it very clear that I am aware of the problems that some residual plots illustrate.

### Assists per Game (AST column) Model

In [ ]:
"""print("Columns used before:", player_data_df.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,12]].columns.values)
print("Columns used in the final version:", player_data_df.iloc[:,[3,5,6,10]].columns.values)"""
player_data_ast=player_data_df.iloc[:,[3,5,6,10]]
X=PolynomialFeatures(degree=2).fit_transform(player_data_ast)
y=player_data_df.AST

X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)
feat_selector=RFECV(LinearRegression(), cv=5).fit(X_train, y_train)

X_train, X_test, y_train, y_test=train_test_split(X[:,feat_selector.support_], y, test_size=0.3, random_state=42)
clf_ast=LinearRegression().fit(X_train, y_train)
y_pred=clf_ast.predict(X_test)

performance_prints(clf_ast, X_train, X_test, y_train, y_test)
my_resid_plot(y_test, y_pred, label_name="AST")
#prediction_plot(y_test, y_pred, label_name="AST")

The 'lines' that you we see a result of AST values being registered in 0.1APG increments (0.1, 0.2, 0.3 etc) and this, considering the moderately small scale of values, introduces a certain level of 'discreteness' in the values both visually and numerically. The same will also be true of the STL residual plot, and the BLK residual plot (for the same reasons).

### Total Rebounds per Game (TRB column) Model

In [ ]:
"""print("Columns used before:", player_data_df.iloc[:,[0,1,2,3,4,5,6,8,9,10,12]].columns.values)
print("Columns used in the final version:", player_data_df.iloc[:,[3,5,6,10]].columns.values)"""
X=PolynomialFeatures(degree=2).fit_transform(player_data_df.iloc[:,[3,4,5,6,8,9,10,12]])
y=player_data_df.TRB
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)

clf_trb=LinearRegression().fit(X_train, y_train)
y_pred=clf_trb.predict(X_test)

performance_prints(clf_trb, X_train, X_test, y_train, y_test)
my_resid_plot(y_test, y_pred, label_name="TRB")

### Blocks per Game (BLK column) Model

In [ ]:
"""print("Columns used before:", player_data_df.iloc[:,[0,2,3,5,6,8,10,11]].columns.values)
print("Columns used in the final version:", player_data_df.iloc[:,[3,5,6,10]].columns.values)"""
X=PolynomialFeatures(degree=2).fit_transform(player_data_df.iloc[:,[3,5,6,8,10]])
y=player_data_df.BLK
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)

clf_blk=LinearRegression().fit(X_train, y_train)
y_pred=clf_blk.predict(X_test)

performance_prints(clf_blk, X_train, X_test, y_train, y_test)
my_resid_plot(y_test, y_pred, label_name="BLK")

###  Steals per Game (STL column) Model

In [ ]:
"""print("Columns used before:", player_data_df.iloc[:,[0,1,2,3,4,5,6,8,10,11,12]].columns.values)
print("Columns used in the final version:", player_data_df.iloc[:,[3,5,6,10]].columns.values)"""
X=PolynomialFeatures(degree=2).fit_transform(player_data_df.iloc[:,[3,5,6,10]])
y=player_data_df.STL
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)

clf_stl=LinearRegression().fit(X_train, y_train)
y_pred=clf_stl.predict(X_test)

performance_prints(clf_stl, X_train, X_test, y_train, y_test)
my_resid_plot(y_test, y_pred, label_name="STL")

## Undrafted Player Models
### Points per Game


In [ ]:
X=player_data_df_undrafted.iloc[:,[3,5,9,11]].values
y=player_data_df_undrafted["PTS"]

X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)
clf_pts=LinearRegression().fit(X_train, y_train)
y_pred=clf_pts.predict(X_test)

performance_prints(clf_pts, X_train, X_test, y_train, y_test)
my_resid_plot(y_test, y_pred, label_name="PTS")

### Assists per Game

In [ ]:
player_data_ast=player_data_df_undrafted.iloc[:,[3,5,9]]
X=PolynomialFeatures(degree=2).fit_transform(player_data_ast)
y=player_data_df_undrafted.AST

X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)
feat_selector=RFECV(LinearRegression(), cv=5).fit(X_train, y_train)

X_train, X_test, y_train, y_test=train_test_split(X[:,feat_selector.support_], y, test_size=0.3, random_state=42)
clf_ast=LinearRegression().fit(X_train, y_train)
y_pred=clf_ast.predict(X_test)

performance_prints(clf_ast, X_train, X_test, y_train, y_test)
my_resid_plot(y_test, y_pred, label_name="AST")

### Total Rebounds per Game

In [ ]:
X=player_data_df_undrafted.iloc[:,[2,3,4,5,7,8,9,11]]
y=player_data_df_undrafted.TRB
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)

clf_trb=LinearRegression().fit(X_train, y_train)
y_pred=clf_trb.predict(X_test)

performance_prints(clf_trb, X_train, X_test, y_train, y_test)
my_resid_plot(y_test, y_pred, label_name="TRB")

### Blocks per Game

In [ ]:
X=PolynomialFeatures(degree=2).fit_transform(player_data_df_undrafted.iloc[:,[3,5,7,9]])
y=player_data_df_undrafted.BLK
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)

clf_blk=LinearRegression().fit(X_train, y_train)
y_pred=clf_blk.predict(X_test)

performance_prints(clf_blk, X_train, X_test, y_train, y_test)
my_resid_plot(y_test, y_pred, label_name="BLK")

### Steals per Game

In [ ]:
X=PolynomialFeatures(degree=2).fit_transform(player_data_df_undrafted.iloc[:,[3,5,10]])
y=player_data_df_undrafted.STL
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)

clf_stl=LinearRegression().fit(X_train, y_train)
y_pred=clf_stl.predict(X_test)

performance_prints(clf_stl, X_train, X_test, y_train, y_test)
my_resid_plot(y_test, y_pred, label_name="STL")

## Conclusions

### Technical
The regression models for Points and Rebounds are fairly good in terms of predictive accuracy, however models for  Assists, Blocks and Steals (as mentioned earlier) are much less accurate. There are a few reasons for this. The numerical aspect: values are very small and more 'discrete' than continuous (0.1, 0.2, 0.3 etc). Inherent randomness/rarity: these statistics are inherently more difficult to predict due to the 'randomness' in Steals, the rarity in Blocks, and the fact that centers aren't usually the ones assisting (while this isn't exactly true in all cases with the game having evolved alot in the last 20 years, it's still an uncommon ask of a center to be assisting others, leading to some inherent randomness in the values). Fundamentally though, it's either a lack of features (which were not scraped) that might have provided greater models for Assists, Steals and Blocks, or perhaps it's inherently much harder to predict due to the beforementioned reasons.

With regards to models for Points and Rebounds (which performed the best), feature normalization and weighted least squares did not aid in getting rid of heteroscendasticity, however this just means we need to be more careful in our statistical inferences.

### Non-Technical

The models for Points per Game and Rebounds per Game are sufficiently accurate to perhaps use in a real world setting for prediction. However, as mentioned before, given that the use cases of the models might differ (along with specific accuracy requirements) it is left for the user to evaluate the residual plots of the model and make their own conclusions as to if it meets their requirements. Custom functions have been made to make it as easy as possible for the user to get predictions for unseen data and to get confidence intervals for the predictions. Predictions is obviously also possible for Assists, Steals and Blocks, however these models are far less accurate.
